# <center> Project-3. EDA+Feature Engineering
---
## <center> Вводные данные и подготовка к работе
---

Первоначальная версия датасета содержит 17 полей со следующей информацией:

- hotel_address — адрес отеля;

- review_date — дата, когда рецензент разместил соответствующий отзыв;

- average_score — средний балл отеля, рассчитанный на основе последнего комментария за последний год;

- hotel_name — название отеля;

- reviewer_nationality — страна рецензента;

- negative_review — отрицательный отзыв, который рецензент дал отелю;

- review_total_negative_word_counts — общее количество слов в отрицательном отзыв;

- positive_review — положительный отзыв, который рецензент дал отелю;

- review_total_positive_word_counts — общее количество слов в положительном отзыве.

- reviewer_score — оценка, которую рецензент поставил отелю на основе своего опыта;

- total_number_of_reviews_reviewer_has_given — количество отзывов, которые рецензенты дали в прошлом;

- total_number_of_reviews — общее количество действительных отзывов об отеле;

- tags — теги, которые рецензент дал отелю;

- days_since_review — количество дней между датой проверки и датой очистки;

- additional_number_of_scoring — есть также некоторые гости, которые просто поставили оценку сервису, но не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.

- lat — географическая широта отеля;

- lng — географическая долгота отеля.

In [166]:
# Импорты
import pandas as pd
import seaborn as sns
import category_encoders as ce

from sklearn.feature_selection import chi2 # хи-квадрат
from sklearn.feature_selection import f_classif # anova

In [167]:
hotels = pd.read_csv('data/hotels.csv')
display(hotels.head())
display(hotels.info())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

None

In [168]:
hotels['hotel_address'][10]
hotels['tags'][0]

"[' Leisure trip ', ' Couple ', ' Studio Suite ', ' Stayed 2 nights ', ' Submitted from a mobile device ']"

---
## <center> Очистка данных
---

In [169]:
#Задание 4.2 - количество наименований отелей
hotels['hotel_name'].value_counts()

hotel_name
Britannia International Hotel Canary Wharf           3587
Strand Palace Hotel                                  3206
Park Plaza Westminster Bridge London                 3095
Copthorne Tara Hotel London Kensington               2688
DoubleTree by Hilton Hotel London Tower of London    2379
                                                     ... 
Ibis Styles Milano Palmanova                            7
Renaissance Paris Republique Hotel Spa                  7
Hotel Wagner                                            6
Hotel Gallitzinberg                                     6
Mercure Paris Porte d Orleans                           5
Name: count, Length: 1492, dtype: int64

In [170]:
#Приведём даты к формату datetime
hotels['review_date'] = pd.to_datetime(hotels['review_date']).dt.date

#Задание 4.3
print('Дата самого первого отзыва:\n',
      hotels['review_date'].sort_values().head(1))
print('Дата самого свежего отзыва:\n',
      hotels['review_date'].sort_values(ascending=False).head(1))

Дата самого первого отзыва:
 143997    2015-08-04
Name: review_date, dtype: object
Дата самого свежего отзыва:
 268933    2017-08-03
Name: review_date, dtype: object


In [171]:
#Приведём столбец days_since_review к удобоваримому 
#для модели виду, убрав 'day' в конце

def day_transf(table_string):
    
    """ Функция-преобразователь для преобразования
    столбца 'days_since_review'.

    Args:
        table_string (_string_): принимает на вход 
        строку из столбца

    Returns:
        _string_: возвращает преобразованную строку
    """    
    
    word_list = table_string.split(' ')
    
    return int(word_list[0])

hotels['days_since_review'] = hotels['days_since_review'].apply(day_transf)

#Проверим результат
hotels['days_since_review'].head(3)

0    531
1    203
2    289
Name: days_since_review, dtype: int64

In [172]:
#Закодируем строковые данные

bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality'])
type_bin = bin_encoder.fit_transform(hotels['reviewer_nationality'])
hotels = pd.concat([hotels, type_bin], axis=1)

In [174]:
# # непрерывные признаки
# num_cols = ['total_number_of_reviews', 'review_total_negative_word_counts', 'additional_number_of_scoring', 'review_total_positive_word_counts', 'total_number_of_reviews_reviewer_has_given', 'days_since_review', 'total_number_of_reviews']

# # категориальные признаки
# cat_cols = ['average_score', 'lat', 'lng', 'hotel_address', 'tags', 'hotel_name', 'reviewer_nationality', 'negative_review', 'positive_review', 'review_date']

In [175]:
# # # !!! Столбцы для первоначального получения MAPE. 
# # # Здесь необходимый минимум приготовлений для 
# # # получения модели !!!

# hotels.info()
# object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
# hotels.drop(object_columns, axis = 1, inplace=True)
# hotels = hotels.fillna(0)
# hotels.info()
# hotels.drop(['lat', 'lng'], axis=1)

In [176]:
# # Разбиваем датафрейм на части, необходимые для обучения и тестирования модели  
# # Х - данные с информацией об отелях, у - целевая переменная (рейтинги отелей)  
# X = hotels.drop(['reviewer_score'], axis = 1)  
# y = hotels['reviewer_score'] 


In [177]:
# imp_cat = pd.Series(chi2(X[cat_cols], y)[0], index=cat_cols)
# imp_cat.sort_values(inplace = True)
# imp_cat.plot(kind = 'barh')

In [178]:
# imp_num = pd.Series(f_classif(X[num_cols], y)[0], index = num_cols)
# imp_num.sort_values(inplace = True)
# imp_num.plot(kind = 'barh')

---
## <center> Обучение модели
---

In [179]:
# # Загружаем специальный инструмент для разбивки:  
# from sklearn.model_selection import train_test_split  
# # Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.  
# # Для тестирования мы будем использовать 25% от исходного датасета.  
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
# # Импортируем необходимые библиотеки:  
# from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
# from sklearn import metrics # инструменты для оценки точности модели  
  
# # Создаём модель  
# regr = RandomForestRegressor(n_estimators=100)  
      
# # Обучаем модель на тестовом наборе данных  
# regr.fit(X_train, y_train)  
      
# # Используем обученную модель для предсказания рейтинга отелей в тестовой выборке.  
# # Предсказанные значения записываем в переменную y_pred  
# y_pred = regr.predict(X_test)  

# # Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они отличаются  
# # Метрика называется Mean Absolute Percentage Error (MAPE) и показывает среднюю абсолютную процентную ошибку предсказанных значений от фактических.  
# print('MAPE:', metrics.mean_absolute_percentage_error(y_test, y_pred))